# Classic text representation techniques

This notebook presents the most typical representation techniques that have been used in natural language processing. These representations are motivated by probability and have traditionally been used as a baseline for text representation. This representation allows us to address common tasks in NLP such as information retrieval, text generation, autocompletion, text classification, topic modeling, among others.


## Bag of Words

The bag-of-words (BoW) is the simplest representation to use. It can be easily calculated for different texts and allows computational calculations to be parallelized, which is useful for large text datasets.

# Classic text representation techniques

<img src="https://miro.medium.com/max/1134/1*lhH8dFbK5_saNe4kcWXwiA.png" width=500>


The formal definition of BoW assumes that there is a finite set of words or vocabulary $\mathcal{V} = \{w_1, w_2, \dots, w_m\}$ and that we have a corpus of $N$ documents $\mathbf{d}_i=[w_a, w_b, \dots, w_d]$ composed of ordered words from the vocabulary $\mathcal{V}$.

Therefore, the BoW representation can be calculated as the categorical distribution of words in a single document $P(w_j|d_i)$, as shown in the following equation:

$$
P(w_j|d_i) \propto \#(w_j \in d_i)\\
P(w_j|d_i) = \frac{\#(w_j \in d_i)}{|d_i|}
$$

This corresponds to the normalized number of occurrences of each word in the document.

For practical examples, we will import some necessary libraries:


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In addition, we will define some help functions:

In [ ]:
def show_bow(X, vocab):
    row_names = [f"d_{i}" for i in range(X.shape[0])]
    return pd.DataFrame(
            data=X,
            columns=vocab,
            index=row_names
            )

### BoW from Scratch

We will implement BoW using Python and Numpy. For this example, we will use some sample texts:

In [ ]:
data = [
        "the black puma chased a rabbit", # each string is a document
        "the rabbit is an animal that is usually fast",
        "the lion is a big cat but a house cat is really different from a lion"
        ] # the complete list is the corpus

First, let's calculate the vocabulary:

In [ ]:
all_text = " ".join(data)
tokens = all_text.split()
vocab = np.unique(tokens)
print(vocab)
print(f"The vocabulary has the following size {vocab.size}")

We will also need a dictionary with the words and tokens:

In [ ]:
indexes = {word: idx for word, idx in zip(vocab, np.arange(vocab.size))}

This means that the BoW will have the following form:

In [ ]:
shape = (len(data), vocab.size)
print(shape)

Let's create a numpy array that will contain the word counts for each document:

In [ ]:
X = np.zeros(shape, dtype=np.float64)

For the calculation, we will define a function that calculates the word counts for a single document.

In [ ]:
def counts_document(d_i, vocab_size=100):
    tokens = d_i.split()
    words, counts = np.unique(
            tokens,
            return_counts=True
            )
    index = [indexes[word] for word in words]
    row = np.zeros((vocab_size, ))
    row[index] = counts
    return row

Now, let's take a look at the complete BoW:

In [ ]:
for i in range(len(data)):
    X[i] = counts_document(
            data[i],
            vocab_size=vocab.size
            )

show_bow(X, vocab)

This result corresponds to the unnormalized BoW. As we will see later, there are different normalization strategies that can be applied to the BoW. For now, in order to have valid probability distributions, we will convert the absolute frequencies into relative frequencies:

In [ ]:
sums = X.sum(axis=0).reshape(1, -1)
BoW = X / sums
show_bow(BoW, vocab)

The above implementation is useful for academic purposes, however, there are Python packages that already do this with high performance and various configurations. In this case, we will explore the `CountVectorizer` from the `sklearn` library.

Let's see how to calculate BoW with this component:

In [ ]:
vect = CountVectorizer().fit(data)
BoW = vect.transform(data).toarray()
show_bow(BoW, vect.get_feature_names_out())

The `CountVectorizer` allows for several filters. We will present some examples using the dataset we will use in this notebook:

This data frame contains tweets related to airlines and their sentiment analysis scores. You can find it at: https://www.kaggle.com/crowdflower/twitter-airline-sentiment.

We can check different properties in this data frame:

In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
import pandas as pd
import io

data = pd.read_csv(io.StringIO(uploaded['Tweets.csv'].decode('utf-8')))

In [ ]:
data.info()

In [ ]:
data.describe(include='O')

In [ ]:
data.head()

In this case, we will focus on the `text` and `airline_sentiment` columns:

In [ ]:
df = data[["text", "airline_sentiment"]].copy()
df

Let's build the BoW representation for this data.

In [ ]:
vect = CountVectorizer().fit(
        df["text"]
        )
vect

Now, the word counts for each tweet would be:

In [ ]:
X = vect.transform(df["text"])
show_bow(
        X.toarray(),
        vect.get_feature_names_out()
        )

As you can see, there are some rare terms that appear once or twice. In addition, there are terms that may appear in every document. We are going to filter these terms, but **why would we want to filter them?**

In [ ]:
vect = CountVectorizer(
        min_df=0.025, # minimum relative frequency to consider a term
        max_df=0.9 # maximum relative frequency to consider a term
        ).fit(
                df["text"]
                )

In [ ]:
X = vect.transform(df["text"])
show_bow(
        X.toarray(),
        vect.get_feature_names_out()
        )

We can also specify a list of stop words that we want to exclude from the BoW:

In [ ]:
vect = CountVectorizer(
        min_df=0.025, # minimum relative frequency to consider a term
        max_df=0.9, # maximum relative frequency to consider a term
        stop_words=["and","about", "after", "again"]
        ).fit(
                df["text"]
                )

In [ ]:
X = vect.transform(df["text"])
show_bow(
        X.toarray(),
        vect.get_feature_names_out()
        )

In [ ]:
doc = data.text.str.cat(sep=' ')

In [ ]:
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                min_font_size = 10).generate(doc)

In [ ]:
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

### Quiz 1:

Include a word cloud here with the 100 most frequently used terms.

In [ ]:
# Insert your code

## TF-IDF

There are some words that are too common in the corpus and are not strictly stopwords. These words differ between datasets, and we must find appropriate ways to identify them.

One of the most common approaches is term frequency inverse document frequency (TF-IDF). This is a weighting scheme that aims to weight words by their appearance in different documents. TF-IDF extends BoW by multiplying the frequency of each term $t_j$ by a weight $w_j$ as follows

$$
\text(TFIDF)(t_i|d_j)=\text{TF}(t_i | d_j) w_i
$$

In this way, we assign a lower weight to terms that are common in different documents and a higher weight to rare terms. There are several ways to calculate $w_i$, the most common approach being the *inverse document frequency* $w_{idf}(t_i)$, which is calculated as follows:

$$
w_{idf}(t_i)=1+\log{\frac{N}{1+df(t_i)}}
$$

Where $N$ is the number of documents in the corpus and $df(t_i)$ is the number of documents containing the term $t_i$.

Now, we can calculate a TF-IDF representation using the `TfidfVectorizer`:

In [ ]:
vect = TfidfVectorizer().fit(df["text"])

In [ ]:
X = vect.transform(df["text"])
show_bow(
        X.toarray(),
        vect.get_feature_names_out()
        )

The `TfidfVectorizer` is similar to the `CountVectorizer`, so we can use the same parameters to filter words by frequency:

In [ ]:
vect = TfidfVectorizer(
        min_df=0.025,
        max_df=0.9
        ).fit(df["text"])

In [ ]:
X = vect.transform(df["text"])
show_bow(
        X.toarray(),
        vect.get_feature_names_out()
        )

In addition, we can change the scale of the frequencies using a logarithm (sublinear scaling).

In [ ]:
vect = TfidfVectorizer(
        min_df=0.1,
        max_df=0.9,
        sublinear_tf=True
        ).fit(df["text"])

In [ ]:
X = vect.transform(df["text"])
show_bow(
        X.toarray(),
        vect.get_feature_names_out()
        )

### Quiz 2:

Include a word cloud here with the 100 most frequently used terms.

In [ ]:
#your code here
data_frame_tfidf = show_bow(X.toarray(),
                          vect.get_feature_names_out())

In [ ]:
# data_frame_tfidf[:100]

In [ ]:

# top_100_freq =

In [ ]:
#top_100_freq.to_csv('top_100_freq.csv')
#insert your code

## Bag of N-Gramas

Bag-of-N-Grams (BoN) is a representation that extends the BoW representation. It counts sequences of tokens (usually characters or words) of different lengths. For example, we can construct 3-grams for the following text:

<emph>the little apple</emph>

For this sequence, the possible 3-grams are:

``python
“the”, “th ”, “e l”, “ li”, “lit”, “itt”, “ttl”, “tle”, “le ”, “e a”, “ap”, “app”, ‘ppl’, “ple”
```

We will implement this natively in Python. First, we define the input text.

In [ ]:
text = "the little apple"
print(text)


Now, we must divide the text into sequences of a certain length:

In [ ]:
n = 3
grams = [
        text[i: i + n]
        for i in range(len(text) - 2)
        ]
print(grams)


Therefore, the representation of N-grams is calculated by counting the number of occurrences of each unique sequence of characters. Let's look at an example with a text containing repeated 3-grams:

In [ ]:
text = "the man who sold the world to another man"
print(text)

In [ ]:
n = 3
grams = [
        text[i: i + n]
        for i in range(len(text) - 2)
        ]
print(grams)

In [ ]:
n_grams = Counter(grams)
print(n_grams)

In Python, we have other implementations that are much more efficient and can calculate N-grams for real-world applications. Let's look at the example with the airline sentiment dataset:

In [ ]:
vect = CountVectorizer(
        analyzer="char", # this specifies that We want to work at characters level.
        ngram_range=(3, 3) # this specifies the range of sequences to consider, in this case only 3.
        ).fit(df["text"])

Now, we can determine the representation of the tri-grams:

In [ ]:
X = vect.transform(df["text"])
show_bow(X.toarray(), vect.get_feature_names_out())

Note that there is a wide range of sequences of size 3. We can filter some of them in a similar way to the BoW representation, i.e., by filtering out terms that are too common or too rare.

In [ ]:
vect = CountVectorizer(
        analyzer="char", # this specifies that We want to work at characters level.
        ngram_range=(3, 3), # this specifies the range of sequences to consider, in this case only 3.
        min_df=0.025,
        max_df=0.9
        ).fit(df["text"])

In [ ]:
X = vect.transform(df["text"])
show_bow(X.toarray(), vect.get_feature_names_out())

Similarly, we can construct a BoN using sequences of words. For example, let's construct a BoN for sequences of 1, 2, and 3 words:

In [ ]:
vect = CountVectorizer(
        analyzer="word", # this specifies that We want to work at characters level.
        ngram_range=(1, 3), # this specifies the range of sequences to consider, in this case only 3.
        min_df=0.010,
        max_df=0.9
        ).fit(df["text"])

In [ ]:
X = vect.transform(df["text"])
show_bow(X.toarray(), vect.get_feature_names_out())

## Autosuggestions with BoN

This is a typical example based on the probabilistic interpretation of BoN as Markov chains.

The general idea is that we are going to estimate the conditional probability:

$$
P(t_k|t_{k-1}, t_{k-2}, \dots, t_0)
$$

However, using a Markov assumption, we intuitively restrict this conditional probability:

* When we have a first-order Markov chain, the probability $P(t_k|t_{k-1})$ of a single token $t_k$ occurring depends only on the previous token $t_{k-1}$.
* When we have a second-order Markov chain, the probability $P(t_k|t_{k-1}, t_{k-2})$ of occurrence of a single token $t_k$ depends on the last two tokens $t_{k-1}$.

The BoN is related to the previous conditional probability; in fact, a BoN corresponds to the estimation of the joint probability $P(t_k, t_{k-1}, t_{k-2}, \dots)$, which can be used to calculate the conditional probability of a sequence of size $N - 1$:

$$
P(t_k| t_{k-1}, t_{k-2}, \dots) = P(t_k, t_{k-1}, t_{k-2}, \dots)
$$

Let's look at this implementation for auto-suggesting the next character for a specific word:

In [ ]:
text = "Once upon a time, there was a child that tried to climb the highest mountain in the world. \"Mark my words, I'll achieve it.\", he said."

vect = CountVectorizer(
        analyzer="char",
        ngram_range=(3, 3)
        ).fit([text])

In [ ]:
X = vect.transform([text])
df2 = show_bow(X.toarray(), vect.get_feature_names_out())
df2

Now, we can divide the sequences

In [ ]:
counts = df2.to_dict()
counts = {key: val["d_0"] for key, val in counts.items()}
counts

Let's calculate the probabilities:

In [ ]:
probs = {}
for token, count in counts.items():
    if token[:2] not in probs:
        probs[token[:2]] = [(token[-1], count)]
    else:
        probs[token[:2]].append((token[-1], count))

print(probs)

Let's look at some examples:

In [ ]:
def complete_word(word):
    seq = word[-2:]
    seq_probs = sorted(
            probs[seq],
            key=lambda x: x[1],
            reverse=True
            )
    print(seq_probs[0][0])

In [ ]:
complete_word("Onc")

In [ ]:
complete_word("Upo")

In [ ]:
complete_word("tim")

In [ ]:
complete_word("tha")

In [ ]:
complete_word("word")

## Document retrieval

One of the most common uses of TF-IDF is information retrieval. A TF-IDF representation can be used to determine the similarity between different documents.

Let's look at an example:

In [ ]:
vect = TfidfVectorizer(
        sublinear_tf=True,
        min_df=0.025,
        max_df=0.9
        ).fit(df["text"])

In [ ]:
X = vect.transform(df["text"]).toarray()
show_bow(X, vect.get_feature_names_out())

We can calculate the most relevant documents in this way:

In [ ]:
query = "there has been a delay since yesterday"
q_repr = vect.transform([query]).toarray()
print(q_repr)

In this case, we use cosine similarity to determine the similarity of each document to the query:

In [ ]:
sims = cosine_similarity(X, q_repr).flatten()
sims.shape

Finally, let's sort the documents in the corpus by similarity and display the five most similar ones.

In [ ]:
df["similarity"] = sims

for document in df.sort_values(
        by="similarity",
        ascending=False
        ).iloc[:5, 0]:
    print(document)